In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from deepface import DeepFace
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import random
import joblib

def enhance_face(input_image_path, scale_factor=6, denoise_strength=6):
    try:
        img = cv2.imread(input_image_path)
        denoised_img = cv2.fastNlMeansDenoisingColored(
            img, None, denoise_strength, denoise_strength, 7, 21
        )
        upscaled_img = cv2.resize(
            denoised_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR
        )
        return upscaled_img
    except Exception as e:
        print(f"Error enhancing image: {e}")
        return None


def generate_vgg_face_embeddings(images):
    embeddings = []
    for img in images:
        try:
            result = DeepFace.represent(img, model_name="VGG-Face", enforce_detection=False)
            if result and "embedding" in result[0]:
                embeddings.append(result[0]["embedding"])
            else:
                print("No embedding found for this image.")
        except Exception as e:
            print(f"Error processing image: {e}")
    return embeddings


def load_images_and_labels(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = enhance_face(img_path)
            if img is not None:
                images.append(img)
                labels.append(label)
    le = LabelEncoder()
    encoded_labels = le.fit_transform(labels)
    return images, encoded_labels, le


def save_model(clf, label_encoder, model_path, encoder_path):
    joblib.dump(clf, model_path)
    joblib.dump(label_encoder, encoder_path)


def load_model(model_path, encoder_path):
    clf = joblib.load(model_path)
    label_encoder = joblib.load(encoder_path)
    return clf, label_encoder


def train_and_save_model(folder, model_path="model.pkl", encoder_path="label_encoder.pkl"):
    train_images, train_labels, label_encoder = load_images_and_labels(folder)
    train_embeddings = generate_vgg_face_embeddings(train_images)

    combined = list(zip(train_embeddings, train_labels))
    random.shuffle(combined)
    train_embeddings, train_labels = zip(*combined)
    train_embeddings = np.array(train_embeddings)
    train_labels = np.array(train_labels)

    clf = SVC(probability=True)
    clf.fit(train_embeddings, train_labels)

    save_model(clf, label_encoder, model_path, encoder_path)


def evaluate_on_test_image(test_image_path, clf, label_encoder):
    enhanced_image = enhance_face(test_image_path)
    faces = detect_faces_with_mtcnn(enhanced_image)

    recognized_names = set()
    for face in faces:
        embedding = DeepFace.represent(face, model_name="VGG-Face", enforce_detection=False)[0]["embedding"]
        prediction = clf.predict_proba([embedding])[0]
        predicted_label_idx = np.argmax(prediction)
        confidence = prediction[predicted_label_idx]
        predicted_label = label_encoder.inverse_transform([predicted_label_idx])[0]
        recognized_names.add(predicted_label)

    return recognized_names


def detect_faces_with_mtcnn(image):
    detector = MTCNN()
    results = detector.detect_faces(image)
    faces = []
    for result in results:
        x, y, w, h = result["box"]
        face = image[y:y+h, x:x+w]
        faces.append(face)
    return faces


def write_attendance_to_excel(recognized_names, students_file, output_path):
    df = pd.read_excel(students_file)
    df["Attendance"] = df["Name"].apply(lambda x: "P" if x in recognized_names else "A")
    df.to_excel(output_path, index=False)


def main():
    students_file = "/content/DL_CLASS.xlsx"
    folder = "/content/drive/MyDrive/face_data"
    test_image_path = "/content/testing.jpeg"
    model_path = "model.pkl"
    encoder_path = "label_encoder.pkl"
    output_path = "/content/attendance.xlsx"

    if not os.path.exists(model_path) or not os.path.exists(encoder_path):
        train_and_save_model(folder, model_path, encoder_path)

    clf, label_encoder = load_model(model_path, encoder_path)
    recognized_names = evaluate_on_test_image(test_image_path, clf, label_encoder)
    write_attendance_to_excel(recognized_names, students_file, output_path)

    print(f"Attendance saved to {output_path}")


# Run the main function
if __name__ == "__main__":
    main()


1/1 [==============================] - 1s 711ms/step
Attendance saved to /content/attendance.xlsx


In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=9fd4e1c541f461567c921f622aebd08dce0bb958270834bea1f3f3e7ff0122b4
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
